# Sparse Alignment columns

In [1]:
class Contig:
    def __init__(self, name, seq):
        self.name = name
        self.seq = seq
        
    def __repr__(self):
        return '< "%s" %i nucleotides>' % (self.name, len(self.seq))

def read_contigs(input_file_path):
    contigs = []
    current_name = ""
    seq_collection = []

    # Pre-read generates an array of contigs with labels and sequences
    with open(input_file_path, 'r') as streamFASTAFile:
        for read in streamFASTAFile.read().splitlines():
            if read == "":
                continue
            if read[0] == ">":
                # If we have sequence gathered and we run into a second (or more) block
                if len(seq_collection) > 0:
                    sequence = "".join(seq_collection)
                    seq_collection = []  # clear
                    contigs.append(Contig(current_name, sequence))
                current_name = read[1:]  # remove >
            else:
                # collects the sequence to be stored in the contig, constant time performance don't concat strings!
                seq_collection.append(read.upper())

    # add the last contig to the list
    sequence = "".join(seq_collection)
    contigs.append(Contig(current_name, sequence))
    return contigs


In [5]:
from collections import Counter

species = read_contigs('9927_alignment.fasta')
for s in species:
    s.name = s.name[:6]
informative_columns = {}
consensus_sequence = []
for col in range(len(species[0].seq)):
    letters = []
    for entry in species:
        letters.append(entry.seq[col])
    column_seq = ''.join(letters)
    consensusing = Counter(column_seq)
    consensus_sequence.append(consensusing.most_common()[0][0])
    if column_seq != letters[0] * len(species) and col > 200 and col < 1500:
        informative_columns[col] = column_seq
        print(column_seq, col+1)
species.append(Contig('Consen', ''.join(consensus_sequence)))

CCCCCCCCCTTCCCCACCCAACCCCCCCC 210
GGGGGGGGGGGGGGGGGGGGGAGGGGGGG 211
CCCCCCCCCCCGCCCCCCCCCCCCCCCCC 212
AAATAAAAAAAAAAAAAAAAAAAAAAGGA 216
GGGGGGGGGGGAGGGGTTGGGGTGGGGGG 223
GGGGGAGGGGGGGGGGGGGGGGGGGGGGG 226
TTTTTTTTTTTAAAAAAATTTTTTTTTTT 229
TTTTTTTTTTTTTTTTTTT-TTTTTTTTT 232
TTTTTTTTTTTTTTTTTTT-TTTTTTTTT 233
CCCCCCCCCCCCCCCCCCC-CCCCCCCCC 234
TTTTTTTTTTTTTTTTTTT-TTTTTTTTT 235
AAAAAAAAAAAAAAAAAAA-AAAAAAAAA 236
GGGGGGGGGGGGGGGGGGC-GGGGGGGGG 237
TTTTTTTTTTTTTTTTTTT-TTTTTTTTT 238
AAAAAAAAAAAAAAAAAAA-AAAAAAAAA 239
CCCCCCCCCCCCCCCCCCC-CCCCCCCCC 240
CCCCCCCCCCCCCCCACCC-CCCCCCCCC 241
AAAAAAAAAAAAAAAAAAA-AAAAAAAAA 242
GGGGGGGGGGGGGGGGGGG-GGGGGGGGG 243
TTTTTTTTTTTTTTTTTTT-TTTTTTTTT 244
AAAAAAAAAAAAAAAAAAA-AAAAAAAAA 245
GGGGGGGGGGGGGGGGGGG-GGGGGGGGG 246
TTTTTTTTTTTTCTTTTTT-TTTCTTTTT 247
GGGGGGGGGGGGGGGGGGG-AGGGGGGGG 248
TTTTTTTTTTTTTTTTTTT-TTTTTTTTT 249
GGGGGGGGGGGGGGGGGGG-GGGGGGGGG 250
GGGGGGGGGGGGGGGGGGG-GGGGGGGGG 251
AAAAAAAAAAAAAAAAAAA-AAAAAAAAA 252
AAAGAAAAAAAAAAAAAAAAAAAAAAAAA 280
CCCCCCCCCCCCCC

* Generate a fasta with informative columns
* Majority vote consensus sequence, but it includes gaps
* transpose?
* CSV file write

In [49]:
with open('9927_informative_positions.csv', 'w') as csv_out:
    csv_out.write('Positions,' + ','.join([str(x+1) for x in sorted(informative_columns.keys())]))
    csv_out.write('\n')
    for entry in species:
        csv_out.write(entry.name[:6] + ",")
        for col in range(len(species[0].seq)):
            if col in informative_columns:
                csv_out.write(entry.seq[col] + ",")
        csv_out.write('\n')
            

## Pair wise table
How well can you differentiate between every species?

In [13]:
seq_length = len(species[0].seq)
similarity_scores = {}
for target in species:
    for query in species:
        if target != query:
            name = (target.name, query.name)
            score = sum([target.seq[i] != query.seq[i] for i in range(250,1500)])
            similarity_scores[name] = score
min(similarity_scores)

('Consen', 'FRAEX3')

In [14]:
with open('9927_differentiability.csv', 'w') as csv_out:
    csv_out.write(',' + ','.join([s.name for s in species]))
    for target in species: # rows
        csv_out.write(target.name +',')
        for query in species: # cells
            if target != query:
                name = (target.name, query.name)
                csv_out.write(str(similarity_scores[name]) + ',')
            else:
                csv_out.write(',')
        csv_out.write('\n')    

In [18]:
min(similarity_scores.values())

0

In [24]:
for k,v in similarity_scores.items():
    if v < 4:
        print(','.join(k))

FRAX09,FRAX10
FRAX15,FRAX16
FRAX15,FRAX01
FRAX16,FRAX15
FRAX01,FRAX16
FRAX16,FRAX01
FRAX01,FRAX15
FRAX10,FRAX09


* Iterate over all the sequences at the same time
* for each position, how many species can you differentiate
* keep of list of species 

# Other Work

In [11]:
base_command = "java -cp CONTEXT-.jar uk.ac.qmul.sbcs.evolution.convergence.runners.BasicAlignmentStats "
data_directory = './Data/'

In [12]:
from glob import glob

for filename in glob(data_directory + '*'):
    print(base_command + filename)

java -cp CONTEXT-.jar uk.ac.qmul.sbcs.evolution.convergence.runners.BasicAlignmentStats ./Data\OG100_full_length_guidance_results_full_length_MUSCLE.MSA.MUSCLE.Without_low_SP_Col.With_Names.fasta
java -cp CONTEXT-.jar uk.ac.qmul.sbcs.evolution.convergence.runners.BasicAlignmentStats ./Data\OG133_full_length_guidance_results_full_length_MUSCLE.MSA.MUSCLE.Without_low_SP_Col.With_Names.fasta
java -cp CONTEXT-.jar uk.ac.qmul.sbcs.evolution.convergence.runners.BasicAlignmentStats ./Data\OG149_full_length_guidance_results_full_length_MUSCLE.MSA.MUSCLE.Without_low_SP_Col.With_Names.fasta
java -cp CONTEXT-.jar uk.ac.qmul.sbcs.evolution.convergence.runners.BasicAlignmentStats ./Data\OG237_full_length_guidance_results_full_length_MUSCLE.MSA.MUSCLE.Without_low_SP_Col.With_Names.fasta
java -cp CONTEXT-.jar uk.ac.qmul.sbcs.evolution.convergence.runners.BasicAlignmentStats ./Data\OG267_full_length_guidance_results_full_length_MUSCLE.MSA.MUSCLE.Without_low_SP_Col.With_Names.fasta
java -cp CONTEXT-.ja

# Processing FinchTV FASTA outputs into and alignment
* Example output FASTA file with all the N's and ugliness
* Load up consensus MSA
* Initial cleanup (lenient)
    * First 50bp are low accuracy => trim them
* Force Alignment onto MSA
    * Do not allow Consensus to have indels introduced, so that we can continue to use the same coordinates

## Notes from Jan Kim
* Heterozygosity inside a single PCR product is a problem because any bias in amplification will exponentially become dominant.
    * PCR as few steps as possible
    * Don't expect perfect 50/50 spilts in the ABI trace
* Geneous (or manual) base caller with ambiguity codes for all significant peaks, not just the tallest
* Do much of this manually, there are only 40 specimens
* Possibly use pairwise alignments that are ambiguity aware to check for best species match
* You could also still use a frozen Multiple Sequence Alignment